In [1]:
from brutus.executors import Lambda
from brutus.aws.lambda_backend.lambda_setup.build_lambda_env import LambdaEnvBuilder
from boto3.session import Session

session = Session(profile_name='default')

executor = Lambda.create(function_name='default-function', 
                         boto3_session=session, 
                         python_packages=['cloudpickle', 'numpy'])
executor

Creating CloudFormation Stack, this can take up to 10mins
Stack Status: CREATE_COMPLETEESS
Successfully validated function!
Clearning lambda_env target dir.
done.
Building environment in Amazon Linux container...
Creating zip...
done


In [2]:
LambdaEnvBuilder.build(['cloudpickle'])

Clearning lambda_env target dir.
done.
Building environment in Amazon Linux container...
Creating zip...
done


In [3]:
lr = session.client('lambda')

with open('brutus/aws/lambda_backend/lambda_env/lambda_env.zip', 'rb') as code:
    code = code.read()

lr.update_function_code(FunctionName=executor.function_name,
                        ZipFile=code,
                        Publish=True)

{'CodeSha256': '+jJw0cDqFVpk7lx6p5JLsIoRqnAjZtpj0tAi4XjRpj8=',
 'CodeSize': 30447,
 'Description': "Function for the Python 'brutus' framework, created by CloudFormation",
 'FunctionArn': 'arn:aws:lambda:eu-west-1:755632011865:function:default-function:6',
 'FunctionName': 'default-function',
 'Handler': 'index.handler',
 'LastModified': '2017-12-29T15:04:54.179+0000',
 'MemorySize': 3000,
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '622',
   'content-type': 'application/json',
   'date': 'Fri, 29 Dec 2017 15:04:54 GMT',
   'x-amzn-requestid': 'a0476115-eca9-11e7-b02b-f5a535e41fa3'},
  'HTTPStatusCode': 200,
  'RequestId': 'a0476115-eca9-11e7-b02b-f5a535e41fa3',
  'RetryAttempts': 0},
 'Role': 'arn:aws:iam::755632011865:role/BrutusRole',
 'Runtime': 'python3.6',
 'Timeout': 300,
 'TracingConfig': {'Mode': 'PassThrough'},
 'Version': '6'}

In [4]:
lr.update_function_configuration(FunctionName=executor.function_name,
                                 Handler='lambda_handler.handler')

{'CodeSha256': '+jJw0cDqFVpk7lx6p5JLsIoRqnAjZtpj0tAi4XjRpj8=',
 'CodeSize': 30447,
 'Description': "Function for the Python 'brutus' framework, created by CloudFormation",
 'FunctionArn': 'arn:aws:lambda:eu-west-1:755632011865:function:default-function',
 'FunctionName': 'default-function',
 'Handler': 'lambda_handler.handler',
 'LastModified': '2017-12-29T15:04:54.358+0000',
 'MemorySize': 3000,
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '635',
   'content-type': 'application/json',
   'date': 'Fri, 29 Dec 2017 15:04:54 GMT',
   'x-amzn-requestid': 'a071cc70-eca9-11e7-b027-2f92375599ee'},
  'HTTPStatusCode': 200,
  'RequestId': 'a071cc70-eca9-11e7-b027-2f92375599ee',
  'RetryAttempts': 0},
 'Role': 'arn:aws:iam::755632011865:role/BrutusRole',
 'Runtime': 'python3.6',
 'Timeout': 300,
 'TracingConfig': {'Mode': 'PassThrough'},
 'Version': '$LATEST'}

In [9]:
def foo(a):
    return a + 1

import cloudpickle
import requests

job = cloudpickle.dumps(dict(func=foo, 
                             args=(1,)
                            )
                       )

r = requests.post(executor._api_endpoint, 
                  data=job)
r.content

b'{"message": "Internal server error"}'

In [15]:
str(job)

"b'\\x80\\x04\\x959\\x01\\x00\\x00\\x00\\x00\\x00\\x00}\\x94(\\x8c\\x04func\\x94\\x8c\\x17cloudpickle.cloudpickle\\x94\\x8c\\x0e_fill_function\\x94\\x93\\x94(h\\x02\\x8c\\x0f_make_skel_func\\x94\\x93\\x94h\\x02\\x8c\\r_builtin_type\\x94\\x93\\x94\\x8c\\x08CodeType\\x94\\x85\\x94R\\x94(K\\x01K\\x00K\\x01K\\x02KCC\\x08|\\x00\\x00d\\x01\\x00\\x17S\\x94NK\\x01\\x86\\x94)\\x8c\\x01a\\x94\\x85\\x94\\x8c\\x1e<ipython-input-9-c749d79e0e9c>\\x94\\x8c\\x03foo\\x94K\\x01C\\x02\\x00\\x01\\x94))t\\x94R\\x94J\\xff\\xff\\xff\\xff}\\x94\\x87\\x94R\\x94}\\x94(\\x8c\\x0eclosure_values\\x94N\\x8c\\x08qualname\\x94h\\x11\\x8c\\x08defaults\\x94N\\x8c\\x04dict\\x94}\\x94\\x8c\\x06module\\x94\\x8c\\x08__main__\\x94\\x8c\\x07globals\\x94}\\x94utR\\x8c\\x04args\\x94K\\x01\\x85\\x94u.'"